# 텐서와 연산

이 노트북은 다음 내용을 다룬다.
* 필요한 패키지를 가져온다.
* 텐서를 만들고 사용한다.
* GPU 가속을 사용한다.
* ```tf.data.Dataset```으로 데이터 파이프라인을 구축한다.

In [1]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import tensorflow as tf
config = tf.compat.v1.ConfigProto(
gpu_options=tf.compat.v1.GPUOptions(
    per_process_gpu_memory_fraction=0.8
))
        
sess = tf.compat.v1.Session(config=config)

## 1. 텐서

텐서는 다차원 배열이다. NumPy의 ```ndarray``` 객체와 유사하게 ```tf.Tensor``` 객체에는 데이터 유형과 형상이 있다. 또한 ```tf.Tensor```는 가속기 메모리에 상주할 수 있다.<br/>
TensorFlow는 ```tf.Tensor```를 소비하고 생성하는 풍부한 라이브러리 연산을 제공한다.(```tf.math.add```, ```tf.linalg.matmul```, ```tf.linalg.inv``` 등)<br/>
이러한 연산은 내장 Python 유형을 자동으로 변환한다.

In [3]:
import tensorflow as tf

tf.Tensor(3, shape=(), dtype=int32)


In [4]:
print(tf.math.add(1, 2))
print(tf.math.add([1, 2], [3, 4]))
print(tf.math.square(5))
print(tf.math.reduce_sum([1, 2, 3]))
print(tf.math.square(2) + tf.math.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


각각 ```tf.Tensor``` 모양과 데이터 타입을 가지고 있다.

In [5]:
x = tf.linalg.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


NumPy 배열과 ```tf.Tensor```의 차이는 다음과 같다.
1. 텐서는 가속기 메모리(GPU, TPU와 같은)에서 사용할 수 있다.
2. ```Tensor```는 불변성(immutable)을 가진다.

텐서는 ```.numpy()``` 메서드를 호출하여 NumPy 배열로 변환할 수 있다.<br/>
가능한 경우, ```tf.Tensor```와 배열은 메모리 표현을 공유하기 때문에 이러한 변환은 일반적으로 저렴하다.<br/>
그러나 ```tf.Tensor```는 GPU 메모리에 저장될 수 있고, NumPy 배열은 항상 호스트 메모리에 저장되므로, 이러한 변환이 항상 가능한 것은 아니다.<br/>
따라서 GPU에서 호스트 메모리로 복사가 필요하다.

In [11]:
import numpy as np

ndarray = np.ones([3, 3])

print('TensorFlow operations convert numpy arrays to Tensors automatically\n')
tensor = tf.math.multiply(ndarray, 42)
print(tensor)

print("\nAnd NumPy operations convert Tensors to NumPy arrays automatically\n")
print(np.add(tensor, 1))

print("\nThe .numpy() method explicitly converts a Tensor to a numpy array\n")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically

tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)

And NumPy operations convert Tensors to NumPy arrays automatically

[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]

The .numpy() method explicitly converts a Tensor to a numpy array

[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## 2. GPU 가속

대부분의 TensorFlow 연산은 GPU를 사용하여 가속화된다.<br/>
어떠한 코드를 명시하지 않아도, TensorFlow는 연산을 위해 CPU 또는 GPU를 사용할 것인지를 자동으로 결정한다.<br/>
필요 시 텐서를 CPU와 GPU 메모리 사이에서 복사한다. 연산에 의해 생성된 텐서는 전형적으로 연산이 실행된 장치의 메모리에 의해 실행된다.

In [12]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is the Tensor on GPU #0:  
True
